In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL
import os
(x_train,___), (_, __) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape([-1,784])
x_train = x_train/255
x_train = x_train.astype(np.float32)

def save_img(hypothesis, epoch):
    n = 0
    hypothesis = hypothesis.reshape([-1,28,28])
    dir = "D:\\ML\\GAN_mnist\\epoch" + str(epoch)
    os.makedirs(dir, exist_ok=True)
    for x in hypothesis:
        img = x*255
        img = img.astype(np.uint8)

        plt.imsave(dir+"\\"+str(n)+".jpg",img,cmap='gray')

        n+=1



class G:
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal([100,256],stddev=0.01))
        self.b1 = tf.Variable(tf.zeros([256]))

        self.W2 = tf.Variable(tf.random.normal([256,784],stddev=0.01))
        self.b2 = tf.Variable(tf.zeros([784]))
        
        self.params = [self.W1, self.b1, self.W2, self.b2]

    def forward(self, z):
        L1 = tf.nn.relu((tf.matmul(z,self.W1)+self.b1))
        L2 = tf.nn.sigmoid((tf.matmul(L1,self.W2)+self.b2))

        return L2

class D:
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal([784,256],stddev=0.01))
        self.b1 = tf.Variable(tf.zeros([256]))

        self.W2 = tf.Variable(tf.random.normal([256,1],stddev=0.01))
        self.b2 = tf.Variable(tf.zeros([1]))


        
        self.params = [self.W1, self.b1, self.W2, self.b2]

    def forward(self, x, dropout=False):
        L1 = tf.nn.relu((tf.matmul(x,self.W1)+self.b1))
        if dropout:
            L1 = tf.nn.dropout(L1, 0.5)
        L2 = (tf.matmul(L1,self.W2)+self.b2)

      
        return L2
epoch_num=100
batch_size = 100
batch_num = int(len(x_train)/batch_size)
print("batchnum", batch_num)

g = G()
d = D()


In [ ]:
for epoch in range(epoch_num+1):
    g_loss_m, d_loss_m = 0,0
    for batch in range(batch_num):
    
        x = x_train[batch*batch_size:(batch+1)*batch_size]
        z = np.random.randn(batch_size,100)*0.1
        z = z.astype(np.float32)
 
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            
            G_z = g.forward(z)
    
            D_x = d.forward(x, dropout=True)
            D_Gz = d.forward(G_z, dropout=True)
            
            cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
            
            d_loss = cross_entropy(tf.ones_like(D_x),D_x) + cross_entropy(tf.zeros_like(D_Gz),D_Gz)
            g_loss = cross_entropy(tf.ones_like(D_Gz),D_Gz)
           
 
        g_grads = g_tape.gradient(g_loss, g.params)
        d_grads = d_tape.gradient(d_loss, d.params)

        g_optimizer = tf.optimizers.Adam(learning_rate=0.0002)
        d_optimizer = tf.optimizers.Adam(learning_rate=0.0002)
        d_optimizer.apply_gradients(grads_and_vars=zip(d_grads, d.params))
        g_optimizer.apply_gradients(grads_and_vars=zip(g_grads, g.params))
        g_loss_m += g_loss/batch_num
        d_loss_m += d_loss/batch_num
        
    if(epoch%1==0):
        print(epoch, g_loss_m.numpy(), d_loss_m.numpy())
        #img = G_z.numpy()[0].reshape([28,28])
        #plt.imshow(img, cmap='Greys', interpolation='nearest')
        #plt.show()
        print("D(G(z)) : ", D_Gz.numpy()[0][0])
        sample =  np.random.randn(10,100)*0.1
        sample = sample.astype(np.float32)
        sample = g.forward(sample)
        if(epoch%10==0): save_img(sample.numpy(), epoch)


0 4.538877 0.3088493
D(G(z)) :  -2.9094274
1 4.6632657 0.075728275
D(G(z)) :  -7.2133703
2 6.868697 0.014964705
D(G(z)) :  -5.052713
3 7.6857457 0.01750085
D(G(z)) :  -6.4765463
4 5.130746 0.07748139
D(G(z)) :  -1.4365313
5 2.4592948 0.19494532
D(G(z)) :  -0.13750714
6 2.8082447 0.24511428
D(G(z)) :  -5.729853
7 3.7452316 0.060840055
D(G(z)) :  -4.207863
8 4.9599705 0.047865875
D(G(z)) :  -6.569439
9 4.425641 0.08997949
D(G(z)) :  -5.6534314
10 4.7909207 0.060263015
D(G(z)) :  -6.248981
11 4.172142 0.09603676
D(G(z)) :  -5.5446124
12 4.5327415 0.074688286
D(G(z)) :  -5.2456684
13 4.645506 0.07230703
D(G(z)) :  -6.6934376
14 4.0612726 0.12741598
D(G(z)) :  -8.514118
15 4.8238254 0.11952338
D(G(z)) :  -3.4043937
16 4.701624 0.11045742
D(G(z)) :  -4.1019263
17 4.824501 0.09510945
D(G(z)) :  -3.528036
18 4.0925956 0.11128934
D(G(z)) :  -6.149585
19 4.1979227 0.11122988
D(G(z)) :  -4.758471
20 4.172768 0.092388846
D(G(z)) :  -6.3947277
21 4.464315 0.07853343
D(G(z)) :  -6.798167
22 4.495427